# Jaylen Hicks
## Data in Hand

In [1]:
#Reference - https://www.youtube.com/watch?v=2mSwcRb3KjQ
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from googleapiclient.discovery import build 
API_KEY = os.environ["API_KEY"]
youtube = build('youtube', 'v3', developerKey=API_KEY)

KeyError: 'API_KEY'

In [3]:
channel_id = "UCp0hYYBW6IMayGgR-WeoCvQ"
ellen_id = "UCp0hYYBW6IMayGgR-WeoCvQ"
colleen_id = "UCrbSjN1_5U7hqFKRA0nvFXQ"
logan_id = "UCG8rbF3g2AMX70yOd8vqIZg"
will_id = "UCKuHFYu3smtrl2AwwMOXOlg"
pew_id = "UC-lHJZR3Gqxm24_Vd_AJ5Yw"

In [4]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()
    
    return response['items']

In [5]:
channel_stats = get_channel_stats(youtube, channel_id)

In [6]:
playlist_id = channel_stats[0]['contentDetails']

In [7]:
def get_video_list(youtube, upload_id):
    video_list = []
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=upload_id,
        maxResults=50
    )
    next_page = True
    while next_page:
        response = request.execute()
        data = response['items']

        for video in data:
            video_id = video['contentDetails']['videoId']
            if video_id not in video_list:
                video_list.append(video_id)

        # Do we have more pages?
        if 'nextPageToken' in response.keys():
            next_page = True
            request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId=upload_id,
                pageToken=response['nextPageToken'],
                maxResults=50
            )
        else:
            next_page = False

    return video_list

In [8]:
channel_stats = get_channel_stats(youtube, channel_id)

In [9]:
playlist_id = channel_stats[0]['contentDetails']['relatedPlaylists']['uploads']
playlist_id

'UUp0hYYBW6IMayGgR-WeoCvQ'

In [10]:
video_list = get_video_list(youtube, playlist_id)

In [11]:
def get_video_details(youtube, video_list):
    stats_list=[]

    # Can only get 50 videos at a time.
    for i in range(0, len(video_list), 50):
        request= youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=video_list[i:i+50]
        )

        data = request.execute()
        for video in data['items']:
            title=video['snippet']['title']
            published=video['snippet']['publishedAt']
            description=video['snippet']['description']
            tag_count= len(video['snippet'].get('tags',[]))
            view_count=video['statistics'].get('viewCount',0)
            like_count=video['statistics'].get('likeCount',0)
            dislike_count=video['statistics'].get('dislikeCount',0)
            comment_count=video['statistics'].get('commentCount',0)
            stats_dict=dict(title=title, description=description, published=published, tag_count=tag_count, view_count=view_count, like_count=like_count, dislike_count=dislike_count, comment_count=comment_count)
            stats_list.append(stats_dict)

    return stats_list

In [12]:
video_data = get_video_details(youtube, video_list)

In [13]:
df=pd.DataFrame(video_data)
df['title_length'] = df['title'].str.len()
df["view_count"] = pd.to_numeric(df["view_count"])
df["like_count"] = pd.to_numeric(df["like_count"])
df["dislike_count"] = pd.to_numeric(df["dislike_count"])
df["comment_count"] = pd.to_numeric(df["comment_count"])
# reaction used later add up likes + dislikes + comments
df["reactions"] = df["like_count"] + df["dislike_count"] + df["comment_count"] + df["comment_count"]
df.head()

,title,description,published,tag_count,view_count,like_count,dislike_count,comment_count,title_length,reactions
0,Greg Kinnear Talks “Stuck on You” in 2003,"[Aired: December 17, 2003]\n\nGreg Kinnear tal...",2023-12-11T16:00:37Z,21,642,45,0,7,42,59
1,Ellen Needs Help Christmas Shopping,"[Aired: December 17, 2003]\n\nEllen wants her ...",2023-12-11T15:00:43Z,21,950,83,0,13,35,109
2,John Stamos Tries the Updog Joke 🐶 #shorts,,2023-12-11T14:00:14Z,21,1952,195,0,7,42,209
3,Ellen Guides Single-Mom Bree on Her Quest for ...,Ellen DeGeneres and Whitney Wolfe Herd team up...,2023-12-09T13:55:00Z,21,12923,130,0,26,64,182
4,Best of Nicki Minaj on the 'Ellen' Show,Nicki Minaj created unforgettable memories on ...,2023-12-08T21:00:09Z,21,38438,2437,0,98,39,2633


In [15]:
df_highest_views = df.nlargest(10, 'view_count')
df_highest_views['title'] = df_highest_views['title'].str[:40]
df_highest_views['view_count_millions'] = df_highest_views['view_count'] / 1000000
df_highest_views

,title,description,published,tag_count,view_count,like_count,dislike_count,comment_count,title_length,reactions,view_count_millions
14294,Nicki Minaj Sings 'Super Bass' with Soph,We were all surprised when we saw Sophia Grace...,2011-10-12T19:59:00Z,26,150382554,971483,0,95894,63,1163271,150.382554
11788,"The Perfect, Last-Minute Kids' Costumes!",Ellen's done it again! She's put together some...,2014-10-29T20:00:02Z,33,127489327,510945,0,0,40,510945,127.489327
10638,Adam Levine's New Girlfriend,Three-year-old Mila is Adam's biggest (little)...,2015-10-16T13:00:02Z,31,116834662,1043636,0,0,28,1043636,116.834662
9050,One-Eyed Monster!,Ellen debuted a brand new game of monstrous pr...,2017-04-07T13:01:00Z,13,92267231,380088,0,14903,17,409894,92.267231
9980,This 3-Year-Old Gymnast Is Flipping Awes,"Emma isn't just an adorable toddler, she's als...",2016-03-24T13:00:01Z,31,91178638,592438,0,0,43,592438,91.178638
10699,Epic or Fail LIVE!,Watch what happened when Ellen and her audienc...,2015-10-01T19:00:01Z,32,82756156,477500,0,0,18,477500,82.756156
11528,Mark Ronson & Bruno Mars Perform 'Uptown,Bruno Mars and Mark Ronson were here for an in...,2015-01-14T14:00:20Z,36,81276461,561647,0,12382,46,586411,81.276461
14218,Sophia Grace and Rosie Sing 'Rolling in,"They're back! And this time, they performed an...",2011-11-09T03:14:35Z,37,72012359,278310,0,40124,49,358558,72.012359
13665,Surprise! Britney Learns 'Gangnam Style',Korean pop star and YouTube sensation Psy gave...,2012-09-11T03:33:08Z,6,65239191,308207,0,31967,50,372141,65.239191
14839,A Huge Surprise for Justin Bieber's Supe,"An Ellen viewer, Paige, is a huge Justin Biebe...",2011-02-09T17:25:16Z,34,65117407,566204,0,55114,45,676432,65.117407


In [17]:
# Most Interations ( Like + Dislikes + Comments)
df_highest_reactions = df.nlargest(10, 'reactions')
df_highest_reactions['title'] = df_highest_reactions['title'].str[:40]
df_highest_reactions['reactions'] = df_highest_reactions['reactions'] / 1000
df_highest_reactions

,title,description,published,tag_count,view_count,like_count,dislike_count,comment_count,title_length,reactions
1973,Then and now: Sophia Grace and Rosie per,,2022-05-22T13:00:12Z,0,62127104,1954322,0,13725,73,1981.772
7448,BTS Get Scared by a Fangirl,K-pop superstars BTS returned to Ellen to talk...,2018-05-25T13:00:00Z,40,51148102,1478947,0,63829,27,1606.605
9300,Ellen's Tribute to the Obamas,To commemorate President Obama's last day in o...,2017-01-19T16:01:30Z,18,62956599,1341465,0,0,29,1341.465
1974,#Diddy talks to Ellen about daughters da,,2022-05-21T13:00:20Z,0,20712429,1207576,0,5755,59,1219.086
14294,Nicki Minaj Sings 'Super Bass' with Soph,We were all surprised when we saw Sophia Grace...,2011-10-12T19:59:00Z,26,150382554,971483,0,95894,63,1163.271
5680,Billie Eilish Surprises Her Fans,"""Bury a Friend"" singer/songwriter Billie Eilis...",2019-06-26T13:00:01Z,23,46735221,1064398,0,24062,32,1112.522
10638,Adam Levine's New Girlfriend,Three-year-old Mila is Adam's biggest (little)...,2015-10-16T13:00:02Z,31,116834662,1043636,0,0,28,1043.636
7622,Cardi B Showed Ellen How She Got Pregnan,Rapper Cardi B visited Ellen for the first tim...,2018-04-19T13:00:02Z,39,51037266,918799,0,23062,41,964.923
9217,Kai's Surprise Star-Studded Performance,The 8-year-old singer caught up with his girlf...,2017-02-14T14:00:00Z,16,53185035,922610,0,0,39,922.610
9648,Ellen and First Lady Michelle Obama Go t,To help prepare the First Lady for life after ...,2016-09-14T13:00:01Z,34,64353903,919433,0,0,45,919.433


In [18]:
# Most Interations ( Like + Dislikes + Comments)
df_highest_reactions = df.nlargest(10, 'reactions')
df_highest_reactions['title'] = df_highest_reactions['title'].str[:40]
df_highest_reactions['reactions'] = df_highest_reactions['reactions'] / 1000
df_highest_reactions

,title,description,published,tag_count,view_count,like_count,dislike_count,comment_count,title_length,reactions
1973,Then and now: Sophia Grace and Rosie per,,2022-05-22T13:00:12Z,0,62127104,1954322,0,13725,73,1981.772
7448,BTS Get Scared by a Fangirl,K-pop superstars BTS returned to Ellen to talk...,2018-05-25T13:00:00Z,40,51148102,1478947,0,63829,27,1606.605
9300,Ellen's Tribute to the Obamas,To commemorate President Obama's last day in o...,2017-01-19T16:01:30Z,18,62956599,1341465,0,0,29,1341.465
1974,#Diddy talks to Ellen about daughters da,,2022-05-21T13:00:20Z,0,20712429,1207576,0,5755,59,1219.086
14294,Nicki Minaj Sings 'Super Bass' with Soph,We were all surprised when we saw Sophia Grace...,2011-10-12T19:59:00Z,26,150382554,971483,0,95894,63,1163.271
5680,Billie Eilish Surprises Her Fans,"""Bury a Friend"" singer/songwriter Billie Eilis...",2019-06-26T13:00:01Z,23,46735221,1064398,0,24062,32,1112.522
10638,Adam Levine's New Girlfriend,Three-year-old Mila is Adam's biggest (little)...,2015-10-16T13:00:02Z,31,116834662,1043636,0,0,28,1043.636
7622,Cardi B Showed Ellen How She Got Pregnan,Rapper Cardi B visited Ellen for the first tim...,2018-04-19T13:00:02Z,39,51037266,918799,0,23062,41,964.923
9217,Kai's Surprise Star-Studded Performance,The 8-year-old singer caught up with his girlf...,2017-02-14T14:00:00Z,16,53185035,922610,0,0,39,922.610
9648,Ellen and First Lady Michelle Obama Go t,To help prepare the First Lady for life after ...,2016-09-14T13:00:01Z,34,64353903,919433,0,0,45,919.433


In [20]:
df_highest_disliked = df.nlargest(10, 'dislike_count')
df_highest_disliked['title'] = df_highest_disliked['title'].str[:40]
df_highest_disliked['dislike_count'] = df_highest_disliked['dislike_count'] / 1000
df_highest_disliked

,title,description,published,tag_count,view_count,like_count,dislike_count,comment_count,title_length,reactions
0,Greg Kinnear Talks “Stuck on You” in 20,"[Aired: December 17, 2003]\n\nGreg Kinnear tal...",2023-12-11T16:00:37Z,21,642,45,0.0,7,42,59
1,Ellen Needs Help Christmas Shopping,"[Aired: December 17, 2003]\n\nEllen wants her ...",2023-12-11T15:00:43Z,21,950,83,0.0,13,35,109
2,John Stamos Tries the Updog Joke 🐶 #shor,,2023-12-11T14:00:14Z,21,1952,195,0.0,7,42,209
3,Ellen Guides Single-Mom Bree on Her Ques,Ellen DeGeneres and Whitney Wolfe Herd team up...,2023-12-09T13:55:00Z,21,12923,130,0.0,26,64,182
4,Best of Nicki Minaj on the 'Ellen' Show,Nicki Minaj created unforgettable memories on ...,2023-12-08T21:00:09Z,21,38438,2437,0.0,98,39,2633
5,Enrique Iglesias' First Appearance on 'E,"[Aired: December 16, 2003]\n\nHe's the 90s hea...",2023-12-08T20:00:23Z,21,7650,132,0.0,9,45,150
6,"Ellen Plays 'Reindeer, Dwarf, or Prescri","[Aired: December 16, 2003]\n\nEnjoy a fun game...",2023-12-08T19:00:19Z,21,2417,57,0.0,3,81,63
7,Shirley MacLaine's First Appearance on ',"[Aired: December 16, 2003]\n\nThe iconic Shirl...",2023-12-08T18:00:14Z,21,9772,156,0.0,7,46,170
8,Season 1: Day 9 of 12 Days of Giveaways,"[Aired: December 16, 2003]\n\nWe continue the ...",2023-12-08T17:00:53Z,21,3990,125,0.0,5,39,135
9,This Christmas Gift Changed John Cena Fo,,2023-12-08T16:00:27Z,21,19328,840,0.0,12,57,864
